# Data Preparing for HW1

My scripts to create the model used in CSB HW1.

In [20]:
import cobra
from cobra import Model, Reaction, Metabolite
import BioModelParser

In [26]:
all_metabolites = BioModelParser.read_metabolites("./iAF1260 Data/iAF1260_metabolites.txt")
m = len(all_metabolites)

In [27]:
all_reactions = BioModelParser.read_reactions("./iAF1260 Data/iAF1260_reactions.txt")
n = len(all_reactions)

In [28]:
# off_reactions = BioModelParser.read_off_reactions("./iAF1260 Data/iAF1260_off_reactions.txt")
off_reactions = BioModelParser.read_off_reactions("./iAF1260 Data/iAF1260_offanaero.txt")

In [29]:
uptake_reactions = BioModelParser.read_uptake_reactions("./iAF1260 Data/iAF1260_uptakes.txt")

In [30]:
stoic_dicts = BioModelParser.read_stoichimetry_matrix("./iAF1260 Data/iAF1260_stoic_matrix.txt")

In [31]:
all_reactions = BioModelParser.update_rxn_types(all_reactions, "./iAF1260 Data/iAF1260_rxn_types.txt")

In [32]:
all_reactions = BioModelParser.modify_bounds(all_reactions, uptake_reactions, off_reactions)

In [33]:
S = BioModelParser.make_stoichiometry_matrix(stoic_dicts, all_reactions, all_metabolites, m, n)

In [34]:
l, u = BioModelParser.make_bounds_vector(all_reactions, n)

In [35]:
model = Model('iAF1260_model');

In [36]:
model_metabolites = []
for met_dict in all_metabolites:
    new_met = Metabolite(met_dict['full_id'], 
                         formula='', 
                         name=met_dict['met_id'], 
                         compartment=met_dict['compartment'])
    model_metabolites.append(new_met)

In [37]:
index = 0
for rxn_dict in all_reactions:
    new_reaction = Reaction(rxn_dict['rxn_id'])
    new_reaction.name = rxn_dict['rxn_id']
    new_reaction.lower_bound = rxn_dict['lower_bound']
    new_reaction.upper_bound = rxn_dict['upper_bound']
    reaction_metabolites = BioModelParser.get_model_metabolite(S[:, index], model_metabolites, m)
    new_reaction.add_metabolites(reaction_metabolites)
    model.add_reactions([new_reaction])
    index += 1

In [38]:
model.objective = 'Ec_biomass_iAF1260_WT_59p81M'

In [39]:
print(model.objective.expression)
print(model.objective.direction)

1.0*Ec_biomass_iAF1260_WT_59p81M - 1.0*Ec_biomass_iAF1260_WT_59p81M_reverse_b0c8b
max


In [40]:
solution = model.optimize()
print(solution)

<Solution 0.931 at 0x7fe1a98278b0>


In [41]:
solution.objective_value

0.9305182477698586

In [42]:
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ca2[e],EX_ca2(e),0.004198,0,0.00%
cbl1[e],EX_cbl1(e),0.0002075,0,0.00%
cl[e],EX_cl(e),0.004198,0,0.00%
cobalt2[e],EX_cobalt2(e),0.002799,0,0.00%
cu2[e],EX_cu2(e),0.002799,0,0.00%
fe2[e],EX_fe2(e),0.006919,0,0.00%
fe3[e],EX_fe3(e),0.006297,0,0.00%
glc-D[e],EX_glc(e),10,0,0.00%
k[e],EX_k(e),0.1574,0,0.00%
mg2[e],EX_mg2(e),0.006997,0,0.00%


In [43]:
cobra.io.write_sbml_model(model, "./iAF1260.xml")